# Spam Detection using LSTM neural network
In this notebook, I will implement a recurrent neural network that perform spam detection. Using an RNN/LSTM rather than a feedforward or logistic regression is more accurate since we can include information about sequence of words.

In [52]:
import tensorflow as tf
import numpy as np
import os
from distutils.version import LooseVersion
import warnings

In [53]:
# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

TensorFlow Version: 1.2.0


# Load data

In [54]:
with open('Assets/SMSSpamCollection', 'r') as f:
    data = f.read()
    
data[:300]

"ham\tGo until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...\nham\tOk lar... Joking wif u oni...\nspam\tFree entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 084528100"

# Preprocessing


In [55]:
# Remove punctuation and lowercase
from string import punctuation
all_text = ''.join([c for c in data if c not in punctuation])
all_text = all_text.lower()

# split label and text of each line.
messages = all_text.split('\n')
messages = [x.split('\t') for x in messages if len(x)>=1]
[labels, texts] = np.array([list(x) for x in zip(*messages)])

In [66]:
print("Example: ")
print("Label: {},\tText: {}".format(labels[0],texts[0]))
print("Label: {},\tText: {}".format(labels[1],texts[1]))
print("Label: {},\tText: {}".format(labels[2],texts[2]))

Example: 
Label: ham,	Text: go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat
Label: ham,	Text: ok lar joking wif u oni
Label: spam,	Text: free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005 text fa to 87121 to receive entry questionstd txt ratetcs apply 08452810075over18s


In [11]:
labels = np.array([1 if each == 'spam' else 0 for each in labels])

In [12]:
labels

array([0, 0, 1, ..., 0, 0, 0])

In [13]:
# All words
all_text = ' '.join(texts)
words = all_text.split()
words[:20]

['go',
 'until',
 'jurong',
 'point',
 'crazy',
 'available',
 'only',
 'in',
 'bugis',
 'n',
 'great',
 'world',
 'la',
 'e',
 'buffet',
 'cine',
 'there',
 'got',
 'amore',
 'wat']

# Encoding the words

In [14]:
from collections import Counter
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse = True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab)}

In [15]:
vocab_to_int

{'apo': 4394,
 'tenerife': 1251,
 'memories': 9083,
 'ayn': 2907,
 'jorgeshock': 4395,
 'siva': 4396,
 'spook': 1601,
 'actual': 3959,
 'ph08700435505150p': 4398,
 'research': 1842,
 'arguing': 9515,
 'bollox': 4399,
 'details': 460,
 'nuerologist': 4400,
 'm8': 2229,
 'satanic': 7912,
 'fifa': 4402,
 'othrs': 4403,
 'eta': 2919,
 'history': 2230,
 'disturbing': 2908,
 'recorded': 8138,
 'mum': 713,
 '2nhite': 8807,
 'arrow': 4407,
 'closed': 2909,
 'dps': 4408,
 'startedindia': 5040,
 'heal': 4410,
 'bridge': 4411,
 'vomit': 1600,
 'nytho': 4412,
 'brotherâ€˜s': 4413,
 'coherently': 4414,
 'bless': 2910,
 'asapok': 4415,
 'grumble': 4416,
 'deluxe': 7693,
 'sounds': 679,
 'calculation': 2911,
 'nigeria': 1140,
 'dips': 4417,
 '9th': 4418,
 'contention': 4419,
 'differbe': 8155,
 'started': 622,
 'hurricanes': 4556,
 'daplease': 9396,
 'dodda': 4424,
 'starting': 1043,
 'uniquei': 4425,
 'glass': 4427,
 'rob': 4428,
 'countinlots': 4429,
 'posh': 9260,
 'guide': 1602,
 'matrix3': 2231,

In [16]:
# Convert the reviews to integers, same shape as reviews list, but with integers
text_ints = []
for each in texts:
    text_ints.append([vocab_to_int[word] for word in each.split()])

In [17]:
text_ints[3]

[5, 234, 142, 23, 360, 3413, 5, 160, 143, 59, 142]

In [18]:
from collections import Counter
text_lens = Counter([len(x) for x in text_ints])
print("Zero-length text: {}".format(text_lens[0]))
print("Maximum text length: {}".format(max(text_lens)))

Zero-length text: 2
Maximum text length: 171


In [19]:
non_zero_idx = [ii for ii, texts in enumerate(text_ints) if len(texts) != 0]
len(non_zero_idx)

5572

In [20]:
len(texts)

5574

In [21]:
# fillter out that review with 0 length
text_ints = [text_ints[ii] for  ii in non_zero_idx]
labels = np.array([labels[ii] for ii in non_zero_idx])

In [22]:
seq_len = 150
features = np.zeros((len(text_ints), seq_len), dtype=int)
for i, row in enumerate(text_ints):
    features[i, -len(row):] = np.array(row)[:seq_len]

In [23]:
features[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,   45,  445,
       5308,  823,  740,  708,   63,    8, 1318,   88,  121,  355, 1384,
        152, 3983, 1388,   67,   57, 7152,  136])

# Training, Validation, Test

In [24]:
split_frac = 0.8
split_idx = int(len(features)*0.8)
train_x, val_x = features[:split_idx], features[split_idx:]
train_y, val_y = labels[:split_idx], labels[split_idx:]

test_idx = int(len(val_x)*0.5)
val_x, test_x = val_x[:test_idx], val_x[test_idx:]
val_y, test_y = val_y[:test_idx], val_y[test_idx:]

print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			Feature Shapes:
Train set: 		(4457, 150) 
Validation set: 	(557, 150) 
Test set: 		(558, 150)


# Build the Neural Network

In [35]:
lstm_size = 150
lstm_layers = 2
batch_size = 250
learning_rate = 0.001
drop_out = 0.5
epochs = 3

Create TF Placeholders for the Neural Network.

In [36]:
n_words = len(vocab_to_int)

# Create the graph object
graph = tf.Graph()
# Add nodes to the graph
with graph.as_default():
    inputs_ = tf.placeholder(tf.int32, [None, None], name='inputs')
    labels_ = tf.placeholder(tf.int32, [None, None], name='labels')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

In [37]:
# Size of the embedding vectors (number of units in the embedding layer)
embed_size = 300 

with graph.as_default():
    embedding = tf.Variable(tf.random_uniform((n_words, embed_size), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, inputs_)

### Build RNN Cell and Initialize
Stack one or more LSTMCells in a MultiRNNCell. (Tensorflow 1.2. if you use tensflow < 1.0, you may get errors)

In [38]:
with graph.as_default():
    def lstm_cell():
        cell = tf.contrib.rnn.LSTMCell(lstm_size, 
                                       initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2),
                                       state_is_tuple=True)
        drop = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)
        return drop
    
    stack_cells = tf.contrib.rnn.MultiRNNCell([lstm_cell() for _ in range(lstm_layers)])
    
    initial_state = state = stack_cells.zero_state(batch_size, tf.float32)

In [39]:
with graph.as_default():
    outputs, final_state = tf.nn.dynamic_rnn(stack_cells, embed, initial_state=initial_state)

In [40]:
with graph.as_default():
    predictions = tf.contrib.layers.fully_connected(outputs[:, -1], 1, activation_fn=tf.sigmoid)
    cost = tf.losses.mean_squared_error(labels_, predictions)
    
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [41]:
with graph.as_default():
    correct_pred = tf.equal(tf.cast(tf.round(predictions), tf.int32), labels_)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [42]:
def get_batches(x, y, batch_size=100):
    
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size], y[ii:ii+batch_size]

In [45]:
with graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    iteration = 1
    for e in range(epochs):
        state = sess.run(initial_state)
        
        for ii, (x, y) in enumerate(get_batches(train_x, train_y, batch_size), 1):
            feed = {inputs_: x,
                    labels_: y[:, None],
                    keep_prob: drop_out,
                    initial_state: state}
            loss, state, _ = sess.run([cost, final_state, optimizer], feed_dict=feed)
            
            if iteration%5==0:
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Train loss: {:.3f}".format(loss))

            if iteration%25==0:
                val_acc = []
                val_state = sess.run(stack_cells.zero_state(batch_size, tf.float32))
                for x, y in get_batches(val_x, val_y, batch_size):
                    feed = {inputs_: x,
                            labels_: y[:, None],
                            keep_prob: 1,
                            initial_state: val_state}
                    batch_acc, val_state = sess.run([accuracy, final_state], feed_dict=feed)
                    val_acc.append(batch_acc)
                print("Val acc: {:.3f}".format(np.mean(val_acc)))
            iteration +=1
    saver.save(sess, "checkpoints/sentiment.ckpt")

Epoch: 0/3 Iteration: 5 Train loss: 0.108
Epoch: 0/3 Iteration: 10 Train loss: 0.086
Epoch: 0/3 Iteration: 15 Train loss: 0.061
Epoch: 1/3 Iteration: 20 Train loss: 0.054
Epoch: 1/3 Iteration: 25 Train loss: 0.029
Val acc: 0.954
Epoch: 1/3 Iteration: 30 Train loss: 0.023
Epoch: 2/3 Iteration: 35 Train loss: 0.033
Epoch: 2/3 Iteration: 40 Train loss: 0.021
Epoch: 2/3 Iteration: 45 Train loss: 0.017
Epoch: 2/3 Iteration: 50 Train loss: 0.025
Val acc: 0.962


In [47]:
test_acc = []
with tf.Session(graph=graph) as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    test_state = sess.run(stack_cells.zero_state(batch_size, tf.float32))
    for ii, (x, y) in enumerate(get_batches(test_x, test_y, batch_size), 1):
        feed = {inputs_: x,
                labels_: y[:, None],
                keep_prob: 1,
                initial_state: test_state}
        batch_acc, test_state = sess.run([accuracy, final_state], feed_dict=feed)
        test_acc.append(batch_acc)
    print("Test accuracy: {:.3f}".format(np.mean(test_acc)))

INFO:tensorflow:Restoring parameters from checkpoints\sentiment.ckpt
Test accuracy: 0.974
